## Gráfico de Barras

Outro tipo de gráfico bastante utilizado é o de barras.

Vamos coletar alguns dados da [Demanda por Vagas - Educação Infantil e EJA
Serie historica da demanda por vagas](http://dados.prefeitura.sp.gov.br/dataset/demanda-por-vagas-educacao-infantil-e-eja)

Primeiro, vamos importar o csv para um Data frame do Pandas.

In [1]:
import pandas as pd

df = pd.read_csv('evolucaodemanda.csv' , 
                 sep = ";", encoding = 'latin-1', header = 0)

ModuleNotFoundError: No module named 'pandas'

Vamos supor que a série histórica de interesse é a referente ao distrito de Água Rasa. Vamos fazer um corte de nosso Data frame, para selecionar apenas as linhas referentes à Água Rasa:

In [ ]:
aguaRasa = df.loc[df['DISTRITO'] == 'AGUA RASA']
aguaRasa = aguaRasa.reset_index(drop=True)

aguaRasa

Vamos criar agora um gráfico de barras, mostrando as matrículas relativas a cada uma das categorias, no último período de que temos dados, dezembro de 2017:

In [ ]:
aguaRasa2017 = aguaRasa.loc[46]
aguaRasa2017

Isso já está bem próximo do que desejamos, mas ainda há algumas linhas que não queremos que apareçam: as que têm zero, as que têm NaN([not a number](https://pt.wikipedia.org/wiki/NaN)) e as que têm DEM_, já que estamos avaliando apenas as matrículas.

In [ ]:
aguaRasa2017 = aguaRasa2017[(aguaRasa2017!=0)].dropna().drop(['DEM_CRECHE'])

aguaRasa2017

Pronto, agora nossa tabela conta apenas com valores válidos.

Vamos agora criar um gráfico a partir dessa tabela.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Criar um vetor de indices
ind = np.arange(len(aguaRasa2017.index[1:-1]))
#Largura das barras
width = 0.35  

fig, ax = plt.subplots()
rects1 = ax.bar(ind,height = aguaRasa2017.values[1:-1], color='SkyBlue')

ax.set_ylabel('Matrículas')
ax.set_title('Matrículas Água Rasa 2017')
ax.set_xticks(ind)
ax.set_xticklabels(aguaRasa2017.index[1:-1], rotation = 45, ha = 'right')
plt.savefig('Exemplos/Matrículas Água Rasa 2017.png', dpi = 300, bbox_inches='tight')



plt.show()


### Série histórica da Demanda

Agora que conseguimos visualizar a demanda em 2017, podemos criar um gráfico com a demanda total de Água Rasa ao longo dos anos.

Vamos primeiro visualizar novamente os dados que temos na variável Água Rasa:

In [ ]:
aguaRasa


### Usando Regex

Na coluna "MES_ANO_REF", temos valores que misturam palavras e números, mais difíceis de trabalhar. Para isso, vamos usar str.replace junto com uma [expressão regular](https://pt.wikipedia.org/wiki/Express%C3%A3o_regular) para remover todos os caracteres que não são números dessa coluna.

In [ ]:
pat = '([a-zA-Z])*\-'
aguaRasa.MES_ANO_REF = aguaRasa.MES_ANO_REF.str.replace( pat, '' )
aguaRasa

Vamos agora remover, com Regex, todas as colunas que começam com "MAT", já que vamos avaliar apenas a demanda.

In [ ]:
pat = '(^MAT)\w+'
aguaRasa.drop(list(df.filter(regex = pat)), axis = 1, inplace = True)
aguaRasa

Estamos quase lá! Vamos agora agrupar os anos pela sua média e verificar como foi a média da demanda.

In [ ]:
#Criar uma lista com todas as colunas contidas em AguaRasa, exceto MES_ANO_REF
col_list= list(aguaRasa)
col_list.remove('MES_ANO_REF')

#Criar uma nova coluna Demanda, que recebe a soma das demandas daquela linha
aguaRasa['Demanda'] = aguaRasa[col_list].sum(axis=1)

#Criar um novo dataframe apenas com as colunas MES_ANO_REF e Demanda 
aguaRasaAno = aguaRasa[['MES_ANO_REF','Demanda']]
#Renomear MES_ANO_REF para Ano
aguaRasaAno = aguaRasaAno.rename(index=str, columns={"MES_ANO_REF": "Ano"})

#Agrupar as linhas por ano e o valor da demanda ser a media daquele ano
aguaRasaAno = aguaRasaAno.groupby(['Ano']).mean()


aguaRasaAno

Vamos criar gráficos, para facilitar a visualização dos dados que obtivemos:

In [ ]:
fig, ax = plt.subplots()

ax.set_ylabel('Demanda')
ax.set_xlabel('Ano')
ax.set_title('Evolução da Demanda - Água Rasa')
ax.plot(aguaRasaAno)


plt.savefig('Exemplos/Evolução da Demanda - Água Rasa 1', dpi = 300)

Modificando as [propriedadas da linha](https://matplotlib.org/1.4.1/users/pyplot_tutorial.html), podemos obter muitas variações deste mesmo gráfico: 

In [ ]:
fig, ax = plt.subplots()

ax.set_ylabel('Demanda')
ax.set_xlabel('Ano')
ax.set_title('Evolução da Demanda - Água Rasa')
line, = ax.plot(aguaRasaAno)
plt.setp(line, color='g', linewidth=2.0, ls = 'steps')


plt.savefig('Exemplos/Evolução da Demanda - Água Rasa 2', dpi = 300)

In [ ]:
fig, ax = plt.subplots()

ax.set_ylabel('Demanda')
ax.set_xlabel('Ano')
ax.set_title('Evolução da Demanda - Água Rasa')
line, = ax.plot(aguaRasaAno)
plt.setp(line, color='black', linewidth=2.0, ls = ':', marker = '+')
for i,j in zip(aguaRasaAno.index.values,list(aguaRasaAno['Demanda'])):
    ax.annotate(str(int(j)),xy=(i, j), color = 'r')

plt.savefig('Exemplos/Evolução da Demanda - Água Rasa 3', dpi = 300)

### Mais um exemplo com Regex

No exemplo do gráfico de barras, utilizamos as informações contidas no csv para criar os nomes no eixo X. Mas os nomes não estão num formato muito agradável. Recuperando esse mesmo exemplo, podemos usar Regex para modificar os textos, a fim de ficarem mais agradáveis:

In [ ]:
aguaRasa2017.index = aguaRasa2017.index.str.replace( "_", ' ' )
aguaRasa2017.index = aguaRasa2017.index.str.replace( "MAT", '' )
aguaRasa2017


In [ ]:


#Criar um vetor de indices
ind = np.arange(len(aguaRasa2017.index[1:-1]))
#Largura das barras
width = 0.35  

fig, ax = plt.subplots()
rects1 = ax.bar(ind,height = aguaRasa2017.values[1:-1], color='SkyBlue')

ax.set_ylabel('Matrículas')
ax.set_xlabel('Séries escolares ')
ax.set_title('Matrículas Água Rasa 2017')
ax.set_xticks(ind)
ax.set_xticklabels(aguaRasa2017.index[1:-1], rotation = 45, ha = 'right')
plt.savefig('Exemplos/Matrículas Água Rasa 2017 - 2.png', dpi = 300, bbox_inches='tight')



plt.show()

## Exportar para CSV